# Context<br/>
### I wanted to understand whether trading the S&P500 index based on the direction of a Stochastic Oscillator indicator consistently at a set opening and closing time could yield a profit. In other words, if the Stochastic Oscillator was facing upward at the specified opening time, I would buy and if it was facing down I would buy. I would then close the trade when the closing time arrived.<br/><br/>Considering that this strategy is purely based on objective facts and no subjective interpretation, calculating this with code is ideal. 

# Problem Statement and Objective<br/>
### Problem Statement
> I need to determine if trading the S&P500 can be profitable with the following trading strategy: opening a trade in the direction of the Stochastic Oscillator at a specified opening time and closing the trade at a set closing time, and doing this consistently over a period of time.
### Objective:
> Create a function with parameters for opening time, closing time, analysis period and analysis symbol. The function should return an output that allows me to determine whether the trading strategy in the problem statement will be profitable, on average, over the analysis period. 

# Methodology<br/>
## Dataset<br/>
### I made use of Yahoo Finance to extract the required dataset. I used this data to manually calculate the %d and %k values for the Stochastic Oscillator.<br/>I then performed a series of pre-processing and formatting steps that would allow me to go through the following steps:<br/>
#### Was the trade successful: If the Stocahstic Osillator was facing upward, did the price also move upward?
#### How big was the price movement if the trade was successful vs unsuccessful: If the losses are larger than the wins, you will lose money
#### What is the probability of having a winning trade or a losing trade: Having a few large winning trades and many small losing trades may be a viable strategy

# Output<br/>
### The output of the calculations is a table that answers the above questions. It specifies the following for wins and losses:
1. How large is the average price movement
2. What is the probability
### This is used to calculate a third column called the expected value which tells us what the expected price move is for a win or loss. We use this to determine if a strategy will make a profit over the long run.<br/>If the expected value of a win is larger than the expected value of a loss, it suggests the strategy will be profitable over the long run.<br/><br/>But how profitable will it be?<br/>The Expected Value Win/Loss Ratio tell us the size of the winning trades relative to the losing trades. The larger the value, the larger the profit. A negative value suggests a loss will be made. A value of zero suggests that you will break even and make no profit.

## Import libraries

In [1]:
# data manipulation
import pandas as pd
import numpy as np

# stock market data import
import yfinance as yf

# Pandas Technical Analysis: for technical analysis tools
import pandas_ta as ta

# date manipulation
from datetime import datetime, timedelta

## Adding my own functions used for filtering data more seamlessly

In [2]:
# Functions to filter data more easily

## filter single column for multiple values - can retain or remove specified values
def filter_more_values(df, col, val:list, neg = False):
    length = len(val)

    if neg == False:
        val1 = df.loc[df[col] == val[0]]
        if length == 1:
            return val1
        else:
            input = []
            input.append(val1)
            for i in range(1, length):
                val2 = df.loc[df[col] == val[i]]
                input.append(val2)
            outputFalse = pd.concat(input)
            return outputFalse

    elif neg == True:
        val1 = df.loc[df[col] != val[0]]
        if length == 1:
            return val1
        else:
            for i in range(1, length):
                val1 = val1.loc[val1[col] != val[i]]
        return val1


def filter_all(df, d:dict):
    for i, v in enumerate(d):
        df = filter_more_values(df, list(d)[i], d.get(v))
    return df

## The primary body of code that formats data and calculates the outputs<br/>
### I cannot spearate this out into different cells as it is included in nested for-loops

In [3]:
output = pd.DataFrame()
o_ts = [8]
c_ts = [10]


open_time = 8
close_time = 20

for open_time in o_ts:

    for close_time in c_ts:

        df = yf.download('ES=F', start='2022-04-01', end='2023-01-01', interval='60m')

        ### Manually calculate stochastic
        # Define periods
        k_period = 14
        d_period = 5
        # Adds a "n_high" column with max value of previous 14 periods
        df['n_high'] = df['High'].rolling(k_period).max()
        # Adds an "n_low" column with min value of previous 14 periods
        df['n_low'] = df['Low'].rolling(k_period).min()
        # Uses the min/max values to calculate the %k (as a percentage)
        df['%K'] = (df['Close'] - df['n_low']) * 100 / (df['n_high'] - df['n_low'])
        # Uses the %k to calculates a SMA over the past 3 values of %k
        df['%D'] = df['%K'].rolling(d_period).mean()

        ### Combined required stochastic and pricing data
        stochastic = df.ta.stoch(high='high', low='low', k=14, d=3, append=True)
        pricing = df[['Open', 'Close']]
        data = pd.concat([pricing, stochastic], axis=1)

        # Rename %k and %d stochastic values 
        data = data.rename(columns={'STOCHk_14_3_3': 'k',
                                    'STOCHd_14_3_3': 'd'})

        data = data.dropna()
        data = data.reset_index()

        # Correct for time difference with South Africa
        data['Datetime'] = data['Datetime'] + timedelta(hours = -7)

        # Create date and hour columns
        data['date'] = [x.date() for x in data['Datetime']]
        data['hour'] = [x.hour for x in data['Datetime']]

        # Keep date, hour, Open, Close, k and d columns for further analysis
        data = data[['date', 'hour', 'Open', 'Close', 'k', 'd']]

        # Filter data to only keep rows with times that are relevant to the analysis - the open and close times 
        data = filter_all(data, {'hour': [open_time, close_time]})

        # Sort dataframe so that the same dates are together and open time always comes before closing time
        data = data.sort_values(by=['date', 'hour'])

        # Shift the open, k and d variables one down for easier calculations to be performed in coming steps
        data['Open'] = data['Open'].shift(1)
        data['k'] = data['k'].shift(1)
        data['d'] = data['d'].shift(1)

        # Filter data to keep only the close time data. The open time data is retained as we shifted it down in the previous step
        data = filter_all(data, {'hour': [close_time]})

        ''' 
        Create new columns with calculations to specify:
                1. price_change: determine if price increased or decreased from open to close; positive=increase, negative=decrease
                2. stochastic_change: determine if stochastic is facing upward or downward at open; positive=upward, negative=downward
                3. price_direction: specify if price increased or decreased from the open time to the closing time; 1=increase, 0=decreased
                4. stochastic_direction: specify if stochastic is facing upward or downward at open; 1=upward, 0=downward 
        '''
        data['price_change'] = data['Close'] - data['Open']
        data['stochastic_change'] = data['k'] - data['d']
        data['price_direction'] = [1 if x > 0 else 0 for x in data['price_change']]
        data['stochastic_direction'] = [1 if x > 0 else 0 for x in data['stochastic_change']]

        # Variable that specifies if the price movement and the direction of the stochastic were the same
        data['agreement'] = [1 if x == y else 0 for x, y in zip(data['price_direction'], data['stochastic_direction'])]

        # Make the price change absolute so that we can compare positive and negative outcomes
        data['price_change'] = data['price_change'].abs()

        # Calculate the probability of the strategy being correct and incorrect; 1=correct, 0=incorrect
        probability = (data.groupby(['agreement'])['agreement'].count()/data['agreement'].count())
        probability = pd.DataFrame(probability).set_index('agreement')
        probability['index'] = [0, 1]
        probability = probability.reset_index()
        probability[['index', 'agreement']]
        probability = probability.rename(columns={'agreement': 'probability'})

        # Calculate the average change in price from open to close for a winning trade and a losing trade; 1=winning trade, 0=losing trade
        avg_trade = data.groupby(['agreement'])['price_change'].mean()
        avg_trade = avg_trade.reset_index()

        # create table that combines the probability figures with the average trade figures
        matrix = pd.concat([avg_trade, probability], axis=1)
        matrix = matrix.iloc[:, [1, 2]]

        # Calculate expected value which specified the expected profit and expected loss for the period
        matrix['expected_value'] = matrix['price_change'] * matrix['probability']

        # Format the table
        matrix = matrix.rename(columns={'price_change': 'Average Price Move'})
        matrix['Outcome'] = ['Loss', 'Win']
        matrix = matrix[['Outcome', 'Average Price Move', 'probability', 'expected_value']]

        # Specify what the open and close time were for this calculation
        print(f'Trade open at {open_time} and closing at {close_time}')

        # Add the open and close time to the final output
        output_data = matrix.copy()
        output_data['open'], output_data['close'] = open_time, close_time

        output = pd.concat([output, output_data])

output

[*********************100%***********************]  1 of 1 completed
Trade open at 8 and closing at 10


,Outcome,Average Price Move,probability,expected_value,open,close
0,Loss,14.125000,0.363636,5.136364,8,10
1,Win,10.857143,0.636364,6.909091,8,10


# Create a function from the above calculations

In [25]:
# 'ES=F'
def calculate_outcomes(open_times:list, close_times:list, start_date:str, end_date:str, symbol:str):

    output = pd.DataFrame()
    o_ts = open_times
    c_ts = close_times

    for open_time in o_ts:

        for close_time in c_ts:

            df = yf.download(symbol, start=start_date, end=end_date, interval='60m')

            ### Manually calculate stochastic
            # Define periods
            k_period = 14
            d_period = 5
            # Adds a "n_high" column with max value of previous 14 periods
            df['n_high'] = df['High'].rolling(k_period).max()
            # Adds an "n_low" column with min value of previous 14 periods
            df['n_low'] = df['Low'].rolling(k_period).min()
            # Uses the min/max values to calculate the %k (as a percentage)
            df['%K'] = (df['Close'] - df['n_low']) * 100 / (df['n_high'] - df['n_low'])
            # Uses the %k to calculates a SMA over the past 3 values of %k
            df['%D'] = df['%K'].rolling(d_period).mean()

            ### Combined required stochastic and pricing data
            stochastic = df.ta.stoch(high='high', low='low', k=14, d=3, append=True)
            pricing = df[['Open', 'Close']]
            data = pd.concat([pricing, stochastic], axis=1)

            # Rename %k and %d stochastic values 
            data = data.rename(columns={'STOCHk_14_3_3': 'k',
                                        'STOCHd_14_3_3': 'd'})

            data = data.dropna()
            data = data.reset_index()

            # Correct for time difference with South Africa
            data['Datetime'] = data['Datetime'] + timedelta(hours = -7)

            # Create date and hour columns
            data['date'] = [x.date() for x in data['Datetime']]
            data['hour'] = [x.hour for x in data['Datetime']]

            # Keep date, hour, Open, Close, k and d columns for further analysis
            data = data[['date', 'hour', 'Open', 'Close', 'k', 'd']]

            # Filter data to only keep rows with times that are relevant to the analysis - the open and close times 
            data = filter_all(data, {'hour': [open_time, close_time]})

            # Sort dataframe so that the same dates are together and open time always comes before closing time
            data = data.sort_values(by=['date', 'hour'])

            # Shift the open, k and d variables one down for easier calculations to be performed in coming steps
            data['Open'] = data['Open'].shift(1)
            data['k'] = data['k'].shift(1)
            data['d'] = data['d'].shift(1)

            # Filter data to keep only the close time data. The open time data is retained as we shifted it down in the previous step
            data = filter_all(data, {'hour': [close_time]})

            ''' 
            Create new columns with calculations to specify:
                    1. price_change: determine if price increased or decreased from open to close; positive=increase, negative=decrease
                    2. stochastic_change: determine if stochastic is facing upward or downward at open; positive=upward, negative=downward
                    3. price_direction: specify if price increased or decreased from the open time to the closing time; 1=increase, 0=decreased
                    4. stochastic_direction: specify if stochastic is facing upward or downward at open; 1=upward, 0=downward 
            '''
            data['price_change'] = data['Close'] - data['Open']
            data['stochastic_change'] = data['k'] - data['d']
            data['price_direction'] = [1 if x > 0 else 0 for x in data['price_change']]
            data['stochastic_direction'] = [1 if x > 0 else 0 for x in data['stochastic_change']]

            # Variable that specifies if the price movement and the direction of the stochastic were the same
            data['agreement'] = [1 if x == y else 0 for x, y in zip(data['price_direction'], data['stochastic_direction'])]

            # Make the price change absolute so that we can compare positive and negative outcomes
            data['price_change'] = data['price_change'].abs()

            # Calculate the probability of the strategy being correct and incorrect; 1=correct, 0=incorrect
            probability = (data.groupby(['agreement'])['agreement'].count()/data['agreement'].count())
            probability = pd.DataFrame(probability).set_index('agreement')
            probability['index'] = [0, 1]
            probability = probability.reset_index()
            probability[['index', 'agreement']]
            probability = probability.rename(columns={'agreement': 'probability'})

            # Calculate the average change in price from open to close for a winning trade and a losing trade; 1=winning trade, 0=losing trade
            avg_trade = data.groupby(['agreement'])['price_change'].mean()
            avg_trade = avg_trade.reset_index()

            # create table that combines the probability figures with the average trade figures
            matrix = pd.concat([avg_trade, probability], axis=1)
            matrix = matrix.iloc[:, [1, 2]]

            # Calculate expected value which specified the expected profit and expected loss for the period
            matrix['expected_value'] = matrix['price_change'] * matrix['probability']

            # Format the table
            matrix = matrix.rename(columns={'price_change': 'Average Price Move'})
            matrix['Outcome'] = ['Loss', 'Win']
            matrix = matrix[['Outcome', 'Average Price Move', 'probability', 'expected_value']]

            # Specify what the open and close time were for this calculation
            print(f'Trade open at {open_time} and closing at {close_time}')

            # Add the open and close time to the final output
            output_data = matrix.copy()
            output_data['open'], output_data['close'] = open_time, close_time

            print(f"Expected Value win/Loss Ratio: {output_data['expected_value'][1] / output_data['expected_value'][0]}")
            
            return output_data

## Test the function with specified parameters

### Results for opening a trade at 8am and closing the trade at 10am

In [26]:
calculate_outcomes([8], [10], '2022-04-01', '2023-02-01', 'ES=F')

[*********************100%***********************]  1 of 1 completed
Trade open at 8 and closing at 10
Expected Value win/Loss Ratio: 1.3451327433628317


,Outcome,Average Price Move,probability,expected_value,open,close
0,Loss,14.125000,0.363636,5.136364,8,10
1,Win,10.857143,0.636364,6.909091,8,10


### Results for opening a trade at 8am and closing the trade at 4pm

In [27]:
calculate_outcomes([8], [16], '2022-04-01', '2023-02-01', 'ES=F')

[*********************100%***********************]  1 of 1 completed
Trade open at 8 and closing at 16
Expected Value win/Loss Ratio: 2.3609660651525446


,Outcome,Average Price Move,probability,expected_value,open,close
0,Loss,16.565068,0.341121,5.650701,8,16
1,Win,20.248214,0.658879,13.341113,8,16


# Conclusion from the above 2 outputs

### The above results suggest that opening a trade at 8am and closing at 10am will see larger price changes on losing trades than on winning trades (On average, a loss will be larger than a win). However, there is a higher probability of a winning trade than a losing trade, so the overall outcome is a higher expected value from winning trades than losing trades - although the expected values are very close together, meaning there will be a very small profit.<br/>The expected Value Win/Loss Ratio tells us that, on average, wins will be 1.35 times larger than losses<br/><br/>On the other hand, opening a trade at 8am and closing it at 4pm suggests that winning trades will have larger price changes than losing trades (On average, a win will be larger than a loss). The probabilities are very similar to the first test, with winning trades being more likely. This results in the expected value for winning trades being more than two times higher than the expected value for losing trades.<br/>The Expected Value Win/Loss Ratio tells us that, on average, wins will be 2.36 time larger than losses.